In [1]:
import numpy as np


# Let's make two numbers with very similar magnitude:

x = 1.48234
y = 1.48235


# Now let's compute their difference in double precision:

x_dbl = np.float64(x)
y_dbl = np.float64(y)
diff_dbl = x_dbl-y_dbl

print(repr(diff_dbl))


# * What would the correct result be?
# * What has happened here?

# -------------
# Can you predict what will happen in single precision?

x_sng = np.float32(x)
y_sng = np.float32(y)
diff_sng = x_sng-y_sng

print(diff_sng)


-1.0000000000065512e-05
-1.001358e-05


In [2]:
import struct
from decimal import Decimal

# Source: https://github.com/mCodingLLC/VideosSampleCode/blob/master/videos/053_0_dot_1__plus__0_dot_2_is_not_0_dot_3_in_most_programming_languages/float_representation.py
# Reference on floats
# https://en.wikipedia.org/wiki/Double-precision_floating-point_format
# https://en.wikipedia.org/wiki/IEEE_754
# https://docs.python.org/3/library/stdtypes.html?highlight=float

def float_to_bin(f) -> str:
    # d for double precision (64 bit) floating point,
    # > for big-endian (the way we usually write numbers in math)
    fmt = ">d"
    bz = struct.pack(fmt, f)
    return "".join(f"{b:08b}" for b in bz)


def every_n_characters(s, n):
    for i in range(0, len(s), n):
        yield s[i:i + n]


def bits_to_float(bits) -> float:
    fmt = ">d"
    bz = bytes([int(chars, base=2) for chars in every_n_characters(bits, 8)])
    t = struct.unpack(fmt, bz)
    return t[0]


def sign_exponent_fraction(s):
    return s[0:1], s[1:12], s[12:64]


def pretty_float_bits(f) -> str:
    return " ".join(sign_exponent_fraction(float_to_bin(f)))


def addition_fail():
    print(".1 + .2:", .1 + .2)
    print(".1 + .2 == .3", .1 + .2 == .3)
    print(f".1:\t\t{pretty_float_bits(.1)}")
    print(f".2:\t\t{pretty_float_bits(.2)}")
    print(f".3:\t\t{pretty_float_bits(.3)}")
    print(f".1+.2:\t{pretty_float_bits(.1 + .2)}")


def one_and_negative_one():
    print(f'1.:\t\t{pretty_float_bits(1.)}')
    print(f'-1.:\t{pretty_float_bits(-1.)}')


def zero_and_negative_zero():
    print(f'0.:\t\t{pretty_float_bits(0.)}')
    print(f'-0.:\t{pretty_float_bits(-0.)}')
    x, y = 0., -0.
    print("0. == -0.?", x == y)
    print("0. is -0.?", x is y)

    x, y = 0, -0
    print("0 is 0?", x is y)


def infinities():
    print(f'2**1023:\t{pretty_float_bits(2. ** 1023)}')
    print(f'2**1024:\t{pretty_float_bits(2. ** 1023 * 2)}')
    print(f'inf:\t\t{pretty_float_bits(float("inf"))}')
    print(f'-inf:\t\t{pretty_float_bits(float("-inf"))}')


def nans():
    print(f'nan:\t\t{pretty_float_bits(float("nan"))}')
    print(f'inf*0.:\t\t{pretty_float_bits(float("inf") * 0.)}')
    my_nan = bits_to_float("0111111111111000000000000001000000000000010000000000000000000000")
    print(f'mynan:\t\t{pretty_float_bits(my_nan)}')

    print("nan == nan?", float("nan") == float("nan"))


def denormalized_numbers():
    print(f'2**-1022:\t{pretty_float_bits(2 ** -1022)}')
    print(f'2**-1023:\t{pretty_float_bits(2 ** -1023)}')
    print(f'2**-1024:\t{pretty_float_bits(2 ** -1024)}')
    print(f'2**-1074:\t{pretty_float_bits(2 ** -1074)}')
    print(f'2**-1075:\t{pretty_float_bits(2 ** -1075)}')
    print(f'2**-1076:\t{pretty_float_bits(2 ** -1076)}')


def almost_equal(x, y, eps=10 ** -6):
    return abs(x - y) < eps


def main():
    addition_fail()

    one_and_negative_one()
    zero_and_negative_zero()
    infinities()
    nans()
    denormalized_numbers()

    print(almost_equal(.1 + .2, .3))

    print(repr(Decimal(".1")))
    print(Decimal(".1") + Decimal(".2") == Decimal(".3"))


if __name__ == '__main__':
    main()


.1 + .2: 0.30000000000000004
.1 + .2 == .3 False
.1:		0 01111111011 1001100110011001100110011001100110011001100110011010
.2:		0 01111111100 1001100110011001100110011001100110011001100110011010
.3:		0 01111111101 0011001100110011001100110011001100110011001100110011
.1+.2:	0 01111111101 0011001100110011001100110011001100110011001100110100
1.:		0 01111111111 0000000000000000000000000000000000000000000000000000
-1.:	1 01111111111 0000000000000000000000000000000000000000000000000000
0.:		0 00000000000 0000000000000000000000000000000000000000000000000000
-0.:	1 00000000000 0000000000000000000000000000000000000000000000000000
0. == -0.? True
0. is -0.? False
0 is 0? True
2**1023:	0 11111111110 0000000000000000000000000000000000000000000000000000
2**1024:	0 11111111111 0000000000000000000000000000000000000000000000000000
inf:		0 11111111111 0000000000000000000000000000000000000000000000000000
-inf:		1 11111111111 0000000000000000000000000000000000000000000000000000
nan:		0 11111111111 10000000

In [3]:

print(repr(1.0))


print(repr(1.0 + 0.00000001))

print(repr(1.0 + 0.0000000000000000000000000000001))


# In[4]:

a = 1.0
while (a + 1.0 > 1.0):
    a = a / 2.0
    print(a)
    
print('-------')
print(2*a)


# In[5]:

print("1.0 + 1.09348570193847509341857e-16")
print(1.0 + 1.09348570193847509341857e-16)


# In[7]:

print("20.0 + 1.09348570193847509341857e-15")
print(20.0 + 1.09348570193847509341857e-15)


# In[8]:

print("1.0 + 1.09348570193847509341857e-15")
print(1.0 + 1.09348570193847509341857e-15)



1.0
1.00000001
1.0
0.5
0.25
0.125
0.0625
0.03125
0.015625
0.0078125
0.00390625
0.001953125
0.0009765625
0.00048828125
0.000244140625
0.0001220703125
6.103515625e-05
3.0517578125e-05
1.52587890625e-05
7.62939453125e-06
3.814697265625e-06
1.9073486328125e-06
9.5367431640625e-07
4.76837158203125e-07
2.384185791015625e-07
1.1920928955078125e-07
5.960464477539063e-08
2.9802322387695312e-08
1.4901161193847656e-08
7.450580596923828e-09
3.725290298461914e-09
1.862645149230957e-09
9.313225746154785e-10
4.656612873077393e-10
2.3283064365386963e-10
1.1641532182693481e-10
5.820766091346741e-11
2.9103830456733704e-11
1.4551915228366852e-11
7.275957614183426e-12
3.637978807091713e-12
1.8189894035458565e-12
9.094947017729282e-13
4.547473508864641e-13
2.2737367544323206e-13
1.1368683772161603e-13
5.684341886080802e-14
2.842170943040401e-14
1.4210854715202004e-14
7.105427357601002e-15
3.552713678800501e-15
1.7763568394002505e-15
8.881784197001252e-16
4.440892098500626e-16
2.220446049250313e-16
1.110223

In [4]:

# coding: utf-8

# In[1]:

x = 0.0

while x != 1.0:
    print(repr(x))
    x = x + 0.1
    
    if x>1.5:
        break


# In[ ]:



0.0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6
0.7
0.7999999999999999
0.8999999999999999
0.9999999999999999
1.0999999999999999
1.2
1.3
1.4000000000000001


In [5]:

# coding: utf-8

# Let's consider $$x^2 + 2px - q = 0$$  We know the roots to be $$x = -p \pm \sqrt{p^2 +q}$$  So let's take a look at $$x = -p + \sqrt{p^2 + q}$$

# Let's take $p$ very large and $q$ to be small:

# In[8]:

from math import sqrt
p = 1e6
q = 0.1

x = -p + sqrt(p**2 + q)
print(repr(x))
print(repr(x**2 + 2*p*x - q))


# Is this accurate?  Not quite.  Let's try rearranging:
# $$
# \frac{q}{p + \sqrt{p^2 + q}}
# $$

# In[9]:

x = q / (p + sqrt(p**2 + q))
print(repr(x))
print(repr(x**2 + 2*p*x - q))


# In[ ]:





4.9942173063755035e-08
-0.00011565387248743675
4.999999999999876e-08
1.3877787807814457e-17
